In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow	import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.metrics import r2_score
from sklearn.compose import ColumnTransformer

In [50]:
data = pd.read_csv('admissions_data.csv')

In [51]:
data.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.472000,107.192000,3.114000,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.295148,6.081868,1.143512,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [52]:
data.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [53]:
features = data.iloc[:, 0:8]
labels = data.iloc[:, -1]

In [54]:
features.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,1,337,118,4,4.5,4.5,9.65,1
1,2,324,107,4,4.0,4.5,8.87,1
2,3,316,104,3,3.0,3.5,8.00,1
3,4,322,110,3,3.5,2.5,8.67,1
4,5,314,103,2,2.0,3.0,8.21,0


In [55]:
labels.head()

0    0.92
1    0.76
2    0.72
3    0.80
4    0.65
Name: Chance of Admit , dtype: float64

In [56]:
features = features.drop(columns=['Serial No.'])

In [57]:
features.shape

(500, 7)

In [58]:
features.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337,118,4,4.5,4.5,9.65,1
1,324,107,4,4.0,4.5,8.87,1
2,316,104,3,3.0,3.5,8.00,1
3,322,110,3,3.5,2.5,8.67,1
4,314,103,2,2.0,3.0,8.21,0


In [59]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, 
                                                                            test_size=0.33, random_state=42) 

In [60]:
features_train

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
471,311,103,3,2.0,4.0,8.09,0
26,322,109,5,4.5,3.5,8.80,0
7,308,101,2,3.0,4.0,7.90,0
453,319,103,3,2.5,4.0,8.76,1
108,331,116,5,5.0,5.0,9.38,1
...,...,...,...,...,...,...,...
106,329,111,4,4.5,4.5,9.18,1
270,306,105,2,2.5,3.0,8.22,1
348,302,99,1,2.0,2.0,7.25,0
435,309,105,2,2.5,4.0,7.68,0


In [61]:
ct = ColumnTransformer([('scale', StandardScaler() , ['GRE Score', 'TOEFL Score'])], remainder='passthrough')

In [62]:
features_train_scale = ct.fit_transform(features_train)

In [63]:
features_test_scale = ct.transform(features_test)

In [65]:
print(features_train.columns)
print(type(features_train_scale))

Index(['GRE Score', 'TOEFL Score', 'University Rating', 'SOP', 'LOR ', 'CGPA',
       'Research'],
      dtype='object')
<class 'numpy.ndarray'>


In [66]:
features_train = pd.DataFrame(features_train_scale, columns = features_train.columns)

In [67]:
features_test = pd.DataFrame(features_test_scale, columns = features_test.columns)

In [68]:
features_train

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,-0.525175,-0.706986,3.0,2.0,4.0,8.09,0.0
1,0.477036,0.276434,5.0,4.5,3.5,8.80,0.0
2,-0.798505,-1.034792,2.0,3.0,4.0,7.90,0.0
3,0.203706,-0.706986,3.0,2.5,4.0,8.76,1.0
4,1.297027,1.423757,5.0,5.0,5.0,9.38,1.0
...,...,...,...,...,...,...,...
330,1.114806,0.604240,4.0,4.5,4.5,9.18,1.0
331,-0.980725,-0.379179,2.0,2.5,3.0,8.22,1.0
332,-1.345165,-1.362599,1.0,2.0,2.0,7.25,0.0
333,-0.707395,-0.379179,2.0,2.5,4.0,7.68,0.0


In [69]:
features_test

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,1.570357,1.423757,4.0,4.0,3.5,9.54,1.0
1,-0.251845,0.112531,4.0,4.5,4.0,9.04,1.0
2,-0.160734,-0.379179,2.0,2.0,2.5,7.65,0.0
3,-0.434065,0.276434,3.0,3.0,3.0,8.69,0.0
4,0.841476,0.768144,3.0,3.5,3.0,9.05,1.0
...,...,...,...,...,...,...,...
160,-1.071835,0.112531,5.0,3.0,3.0,8.48,0.0
161,0.659256,0.604240,3.0,2.5,2.0,8.80,1.0
162,-0.069624,-0.706986,2.0,2.0,4.5,8.74,0.0
163,-0.616285,-1.362599,2.0,1.5,2.0,7.30,0.0


In [94]:
def model_design(features):
    model = Sequential(name="regression_model")
    num_features = features.shape[1]
    input = layers.InputLayer(input_shape=(num_features,))
    model.add(input)
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1)) 
    opt = Adam(learning_rate=0.01)
    model.compile(loss='mse', metrics=['mae'], optimizer=opt)
    return model

In [95]:
model = model_design(features_train)
print(model.summary())

Model: "regression_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 128)               1024      
                                                                 
 dense_11 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1,153
Trainable params: 1,153
Non-trainable params: 0
_________________________________________________________________
None


In [96]:
model.fit(features_train, labels_train, epochs=40, batch_size=1, verbose=1)

Epoch 1/40
335/335 [==============================] - 1s 2ms/step - loss: 0.0720 - mae: 0.1697
Epoch 2/40
335/335 [==============================] - 0s 1ms/step - loss: 0.0118 - mae: 0.0872
Epoch 3/40
335/335 [==============================] - 0s 1ms/step - loss: 0.0084 - mae: 0.0701
Epoch 4/40
335/335 [==============================] - 0s 1ms/step - loss: 0.0077 - mae: 0.0660
Epoch 5/40
335/335 [==============================] - 0s 1ms/step - loss: 0.0068 - mae: 0.0629
Epoch 6/40
335/335 [==============================] - 0s 1ms/step - loss: 0.0066 - mae: 0.0637
Epoch 7/40
335/335 [==============================] - 0s 1ms/step - loss: 0.0061 - mae: 0.0604
Epoch 8/40
335/335 [==============================] - 0s 1ms/step - loss: 0.0076 - mae: 0.0677
Epoch 9/40
335/335 [==============================] - 0s 1ms/step - loss: 0.0071 - mae: 0.0657
Epoch 10/40
335/335 [==============================] - 0s 1ms/step - loss: 0.0066 - mae: 0.0619
Epoch 11/40
335/335 [============================

In [97]:
val_mse, val_mae = model.evaluate(features_test, labels_test, verbose=0)

In [98]:
print(f'Validation MSE: {val_mse}, Validation MAE: {val_mae}')

Validation MSE: 0.0073916371911764145, Validation MAE: 0.0721481516957283


Hyperparameter Adjustment:

In [99]:
model.fit(features_train, labels_train, epochs=5, batch_size=20, verbose=1)

Epoch 1/5
17/17 [==============================] - 0s 2ms/step - loss: 0.0055 - mae: 0.0580
Epoch 2/5
17/17 [==============================] - 0s 3ms/step - loss: 0.0047 - mae: 0.0516
Epoch 3/5
17/17 [==============================] - 0s 2ms/step - loss: 0.0045 - mae: 0.0488
Epoch 4/5
17/17 [==============================] - 0s 2ms/step - loss: 0.0046 - mae: 0.0497
Epoch 5/5
17/17 [==============================] - 0s 2ms/step - loss: 0.0045 - mae: 0.0496


In [100]:
print(f'Validation MSE: {val_mse}, Validation MAE: {val_mae}')

Validation MSE: 0.0073916371911764145, Validation MAE: 0.0721481516957283
